In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
# --- Función de carga y preprocesamiento de datos ---
def carga_datos_csv(data, categoria, porcentaje_test=0.25, hot_encoded=None, normalizados=None, seed=None):
    # Normalización
    if normalizados:
        for col in normalizados:
            if col in data.columns:
                data[col] = (data[col] - data[col].min()) / (data[col].max() - data[col].min())

    # Dividir en train y test
    train, test = train_test_split(data, test_size=porcentaje_test, random_state=seed)

    # Separar etiquetas
    label_train = train[categoria].copy()
    label_test = test[categoria].copy()
    train = train.drop(columns=[categoria])
    test = test.drop(columns=[categoria])

    # One-hot encoding
    if hot_encoded:
        full_data = pd.concat([train, test])
        full_data = pd.get_dummies(full_data, columns=hot_encoded)

        train = full_data.iloc[:len(train), :]
        test = full_data.iloc[len(train):, :]

    # Alinear columnas
    test = test.reindex(columns=train.columns, fill_value=0)

    return train.columns, np.array(train), np.array(label_train), np.array(test), np.array(label_test)

In [ ]:
# --- Configuración ---
dataset = 'dataset_v1.csv'  # Ruta al CSV
n_neighbors = 7
metric = 'euclidean'
label = 'Candidatura Ganadora'

# Columnas categóricas y numéricas
hot = ['Comunidad', 'Provincia', 'Gobierno Actual de Comunidad', 'Gobierno Actual de España']
norm = ['Edad Media', 'Escolarizacion', 'Esperanza de Vida', 'Habitantes Provincia',
        'Acceso a Internet', 'PIB per capita', 'Criminalidad', 'Inmigracion',
        '% de mujeres', 'Tasa de Pobreza', 'Tasa Desempleo']


In [ ]:
# --- Carga y preparación de datos ---
data = pd.read_csv(dataset, sep=',')
data = data.dropna(subset=[label])  # Asegurar que hay etiquetas

x_columns, x_train, y_train, x_test, y_test = carga_datos_csv(
    data, label, porcentaje_test=0.25, hot_encoded=hot, normalizados=norm, seed=int(datetime.utcnow().timestamp())
)


In [ ]:
# --- Resultados ---
np.set_printoptions(threshold=np.inf)

print("\nMatriz de confusión:")
labels = sorted(np.unique(np.concatenate([y_test, y_pred])))
cm = confusion_matrix(y_test, y_pred, labels=labels)
print(cm)

print("\nPrecisión global:")
print(accuracy_score(y_test, y_pred))

print("\nInforme de clasificación:")
print(classification_report(y_test, y_pred, zero_division=0))

In [ ]:
# --- Visualización de la matriz de confusión ---
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=labels, yticklabels=labels)
plt.xlabel('Predicción')
plt.ylabel('Valor real')
plt.title('Matriz de confusión - KNN')
plt.tight_layout()
plt.show()